In [4]:
import math
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from collections import OrderedDict
from functools import reduce
from tqdm import tqdm, trange
from copy import deepcopy

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import difflib

plt.rc('font',family='Times New Roman')
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['font.size'] = 12

In [5]:
#设置value的显示长度为200，默认为50
# pd.set_option('max_colwidth',200)
# #显示所有列，把行显示设置成最大
# pd.set_option('display.max_columns', None)
#显示所有行，把列显示设置成最大
pd.set_option('display.max_rows', 500)

In [6]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# 预测

In [7]:
def read_predict_data(predict_data, batch_size):
    hla_sequence = pd.read_csv('/home/chujunyi/5_ZY_MHC/data/common_hla_sequence.csv')
    
    print('# Samples = ', len(predict_data))
    try:
        predict_data = pd.merge(predict_data, hla_sequence, on = 'HLA')
    except:
        predict_data = pd.merge(predict_data, hla_sequence, on = 'HLA_sequence')
        
    pep_inputs, hla_inputs = make_data(predict_data)
    data_loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs), batch_size, shuffle = False, num_workers = 0)
    return predict_data, pep_inputs, hla_inputs, data_loader

In [8]:
def make_data(data):
    pep_inputs, hla_inputs = [], []
    if 'peptide' not in data.columns: 
        peptides = data.mutation_peptide
    else:
        peptides = data.peptide
        
    for pep, hla in zip(peptides, data.HLA_sequence):
        pep, hla = pep.ljust(pep_max_len, '-'), hla.ljust(hla_max_len, '-')
        pep_input = [[vocab[n] for n in pep]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
        hla_input = [[vocab[n] for n in hla]]
        pep_inputs.extend(pep_input)
        hla_inputs.extend(hla_input)
    return torch.LongTensor(pep_inputs), torch.LongTensor(hla_inputs)

class MyDataSet(Data.Dataset):
    def __init__(self, pep_inputs, hla_inputs):
        super(MyDataSet, self).__init__()
        self.pep_inputs = pep_inputs
        self.hla_inputs = hla_inputs

    def __len__(self): # 样本数
        return self.pep_inputs.shape[0] # 改成hla_inputs也可以哦！

    def __getitem__(self, idx):
        return self.pep_inputs[idx], self.hla_inputs[idx]

In [9]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [10]:
def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.
        
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
        return context, attn
    
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device("cuda" if self.use_cuda else "cpu")
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)
    def forward(self, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        residual, batch_size = input_Q, input_Q.size(0)
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context) # [batch_size, len_q, d_model]
        return nn.LayerNorm(d_model).to(device)(output + residual), attn

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device("cuda" if self.use_cuda else "cpu")
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).to(device)(output + residual) # [batch_size, seq_len, d_model]

In [11]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]
        '''
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn
    
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len]
        enc_self_attns = []
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

In [12]:
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, dec_self_attn_mask): # dec_inputs = enc_outputs
        '''
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        '''
        # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs) # [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attn
    
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
#         self.tgt_emb = nn.Embedding(d_model * 2, d_model)
        self.use_cuda = use_cuda
        device = torch.device("cuda" if self.use_cuda else "cpu")
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])
        self.tgt_len = tgt_len
        
    def forward(self, dec_inputs): # dec_inputs = enc_outputs (batch_size, peptide_hla_maxlen_sum, d_model)
        '''
        dec_inputs: [batch_size, tgt_len]
        enc_intpus: [batch_size, src_len]
        enc_outputs: [batsh_size, src_len, d_model]
        '''
#         dec_outputs = self.tgt_emb(dec_inputs) # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_inputs.transpose(0, 1)).transpose(0, 1).to(device) # [batch_size, tgt_len, d_model]
#         dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        dec_self_attn_pad_mask = torch.LongTensor(np.zeros((dec_inputs.shape[0], tgt_len, tgt_len))).bool().to(device)

        dec_self_attns = []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            dec_outputs, dec_self_attn = layer(dec_outputs, dec_self_attn_pad_mask)
            dec_self_attns.append(dec_self_attn)
            
        return dec_outputs, dec_self_attns

In [13]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device("cuda" if use_cuda else "cpu")
        self.pep_encoder = Encoder().to(device)
        self.hla_encoder = Encoder().to(device)
        self.decoder = Decoder().to(device)
        self.tgt_len = tgt_len
        self.projection = nn.Sequential(
                                        nn.Linear(tgt_len * d_model, 256),
                                        nn.ReLU(True),

                                        nn.BatchNorm1d(256),
                                        nn.Linear(256, 64),
                                        nn.ReLU(True),

                                        #output layer
                                        nn.Linear(64, 2)
                                        ).to(device)
        
    def forward(self, pep_inputs, hla_inputs):
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        pep_enc_outputs, pep_enc_self_attns = self.pep_encoder(pep_inputs)
        hla_enc_outputs, hla_enc_self_attns = self.hla_encoder(hla_inputs)
        enc_outputs = torch.cat((pep_enc_outputs, hla_enc_outputs), 1) # concat pep & hla embedding
        
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        dec_outputs, dec_self_attns = self.decoder(enc_outputs)
        dec_outputs = dec_outputs.view(dec_outputs.shape[0], -1) # Flatten [batch_size, tgt_len * d_model]
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        return dec_logits.view(-1, dec_logits.size(-1)), pep_enc_self_attns, hla_enc_self_attns, dec_self_attns

In [14]:
def eval_step(model, val_loader, use_cuda = False):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        y_prob_val_list, dec_attns_val_list = [], []
        for val_pep_inputs, val_hla_inputs in tqdm(val_loader):
            val_pep_inputs, val_hla_inputs = val_pep_inputs.to(device), val_hla_inputs.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)

            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()
            y_prob_val_list.extend(y_prob_val)
            
            dec_attns_val_list.extend(val_dec_self_attns[0][:, :, 15:, :15]) # 只要（34,15）行HLA，列peptide
                    
        y_pred_val_list = transfer(y_prob_val_list, threshold)
    
    return y_pred_val_list, y_prob_val_list, dec_attns_val_list
        
def transfer(y_prob, threshold = 0.5):
    return np.array([[0, 1][x > threshold] for x in y_prob])

# 参数

In [15]:
pep_max_len = 15 # peptide; enc_input max sequence length
hla_max_len = 34 # hla; dec_input(=dec_output) max sequence length
tgt_len = pep_max_len + hla_max_len

vocab = np.load('Transformer_vocab_dict.npy', allow_pickle = True).item()
vocab_size = len(vocab)

# Transformer Parameters
d_model = 64  # Embedding Size
d_ff = 512 # FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V

batch_size = 1024
threshold = 0.5

model_pwd = '/home/chujunyi/5_ZY_MHC/model/pHLAIformer/'
model_file = 'model_layer1_multihead9_fold4.pkl'
model_layer_head = ['1', '9', '4']
n_layers, n_heads, fold = 1, 9, 4

In [16]:
predict_peptides = ['KIPLRCTA', 'SRELSFTSA', 'GGGLGPVDV', 'AERLQENLQ', 'KLVSIKIQML', 'QVIIFVKSVQR', 'VSPMRQVEPPA', 'NGKPLPGATPAK',  'SLYGGNAVVELVTV', 'AKEMMAQKRK', 'VKKIKEHVRSKTKV']
predict_HLAs = ['HLA-B*27:05','HLA-A*68:01','HLA-A*68:01','HLA-A*68:01','HLA-A*68:01','HLA-A*68:01','HLA-C*16:01','HLA-B*27:02' ,'HLA-A*02:01','HLA-A*02:06','HLA-B*27:05']
predict_data = pd.DataFrame([predict_HLAs, predict_peptides], index = ['HLA', 'peptide']).T

predict_data, predict_pep_inputs, predict_hla_inputs, predict_loader = read_predict_data(predict_data, batch_size)
predict_data

# Samples =  11


,HLA,peptide,HLA_sequence
0,HLA-B*27:05,KIPLRCTA,YHTEYREICAKTDEDTLYLNYHDYTWAVLAYEWY
1,HLA-B*27:05,VKKIKEHVRSKTKV,YHTEYREICAKTDEDTLYLNYHDYTWAVLAYEWY
2,HLA-A*68:01,SRELSFTSA,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY
3,HLA-A*68:01,GGGLGPVDV,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY
4,HLA-A*68:01,AERLQENLQ,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY
5,HLA-A*68:01,KLVSIKIQML,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY
6,HLA-A*68:01,QVIIFVKSVQR,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY
7,HLA-C*16:01,VSPMRQVEPPA,YYAGYREKYRQTDVSNLYLWYDSYTWAAQAYTWY
8,HLA-B*27:02,NGKPLPGATPAK,YHTEYREICAKTDENIAYLNYHDYTWAVLAYEWY
9,HLA-A*02:01,SLYGGNAVVELVTV,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY


In [17]:
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

model_eval = Transformer().to(device)
model_eval.load_state_dict(torch.load(model_pwd + model_file), strict = True)

model_eval.eval()
y_pred, y_prob, attns = eval_step(model_eval, predict_loader, use_cuda)

predict_data['y_pred'], predict_data['y_prob'] = y_pred, y_prob
predict_data = predict_data.round({'y_prob': 4})
predict_data

100%|██████████| 1/1 [00:00<00:00, 63.85it/s]


,HLA,peptide,HLA_sequence,y_pred,y_prob
0,HLA-B*27:05,KIPLRCTA,YHTEYREICAKTDEDTLYLNYHDYTWAVLAYEWY,0,0.0000
1,HLA-B*27:05,VKKIKEHVRSKTKV,YHTEYREICAKTDEDTLYLNYHDYTWAVLAYEWY,0,0.0001
2,HLA-A*68:01,SRELSFTSA,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY,0,0.0000
3,HLA-A*68:01,GGGLGPVDV,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY,0,0.0000
4,HLA-A*68:01,AERLQENLQ,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY,0,0.0000
5,HLA-A*68:01,KLVSIKIQML,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY,0,0.0000
6,HLA-A*68:01,QVIIFVKSVQR,YYAMYRNNVAQTDVDTLYIMYRDYTWAVWAYTWY,1,0.9999
7,HLA-C*16:01,VSPMRQVEPPA,YYAGYREKYRQTDVSNLYLWYDSYTWAAQAYTWY,0,0.0000
8,HLA-B*27:02,NGKPLPGATPAK,YHTEYREICAKTDENIAYLNYHDYTWAVLAYEWY,0,0.0000
9,HLA-A*02:01,SLYGGNAVVELVTV,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY,1,1.0000


In [18]:
len(attns), attns[0].shape

(11, torch.Size([9, 34, 15]))

In [19]:
# torch.save(attn_res, './results/pHLAIformer_{}_data_attn.pt'.format(type_))

# 画注意力图

In [20]:
def pHLA_attns_draw_save(data, attn_data, hla = False, peptide = False, attn_savepath = False, show = False, show_savepath = False):
    
    idx = data[data.HLA == hla][data.peptide == peptide].index[0]
    hla_sequence = list(data.iloc[idx].HLA_sequence)
    y_pred = data.iloc[idx].y_pred
    y_prob = data.iloc[idx].y_prob.round(4)
    length = len(peptide)
    
    for head in trange(9):
        if head == 0:
            temp = deepcopy(attn_data[idx][head][:, :length])
        else:
            temp += deepcopy(attn_data[idx][head][:, :length])

    temp_pd = pd.DataFrame(np.array(temp), index = hla_sequence, columns = list(peptide))
    temp_pd.loc['sum'] = temp_pd.sum(axis = 0)
    temp_pd.loc['posi'] = range(1, length + 1)
    temp_pd.loc['contrib'] = temp_pd.loc['sum'] / temp_pd.loc['sum'].sum()
        
    if attn_savepath: 
        temp_pd.to_save(attn_savepath + '{}_{}_softmax_attention.csv')
        
    ###############################
    
    if show:
        fig = plt.figure(figsize = (10, 3))
        fig.patch.set_facecolor('white')
        
        cmap = 'cividis'
        sns.heatmap(temp_pd.iloc[:-3].T, cmap = cmap, square = True,  
                    xticklabels = True, yticklabels = True, cbar = True)
        plt.ylabel('Peptide')
        plt.xlabel('HLA')
        plt.title('{} | {} | {} | Probability = {}'.format(hla, peptide, ['Negative', 'Positive'][y_pred == 1], y_prob))

        if show_savepath:
            plt.savefig(show_savepath + '{}_{}_{}_Prob{}.tif'.format(hla, peptide, ['Negative', 'Positive'][y_pred == 1], y_prob),
                        dpi = 600, pil_kwargs = {'compression': 'tiff_lzw'}, bbox_inches = 'tight', facecolor=fig.get_facecolor())

        plt.show()
    
    return temp_pd, hla, peptide

### pHLA

In [21]:
def pHLA_attns_keyaatype_keyaacontrib(data, attns, hla = False, peptide = False): 
    # 已知是负样本的情况下
    pHLA_attns_pd, hla, peptide = pHLA_attns_draw_save(data, attns, hla, peptide)

    pHLA_keyaatype = OrderedDict()
    for posi, pep_aa, aa_attn_sum in zip(list(pHLA_attns_pd.loc['posi']),
                                         list(pHLA_attns_pd.columns),
                                         list(pHLA_attns_pd.loc['sum'])):

        pHLA_keyaatype[int(posi)] = [pep_aa, aa_attn_sum]
    pHLA_keyaatype = OrderedDict(sorted(pHLA_keyaatype.items(), key = lambda t: (-t[1][1])))

    pHLA_keyaatype_contrib = OrderedDict()
    for posi, pep_aa, aa_attn_contrib in zip(list(pHLA_attns_pd.loc['posi']),
                                         list(pHLA_attns_pd.columns),
                                         list(pHLA_attns_pd.loc['contrib'])):

        pHLA_keyaatype_contrib[int(posi)] = [pep_aa, aa_attn_contrib]
    pHLA_keyaatype_contrib = OrderedDict(sorted(pHLA_keyaatype_contrib.items(), key = lambda t: (-t[1][1])))

    return pHLA_attns_pd, pHLA_keyaatype, pHLA_keyaatype_contrib

In [22]:
def HLA_length_aatype_position_num(hla = False, length = 9, label = None):

    if label == 'None': 
        new_label = 'all'
    elif label == 1:
        new_label = 'positive'
    elif label == 0:
        new_label == 'negative'
    
    try:
        aatype_position = np.load('./results/Attention/pHLAIformer_{}_Length{}_Label_Unsoftmax_pepaatype_pepposition_pd.npy'.format(hla, length),
                                    allow_pickle = True).item()[new_label]

        aatype_position_num = np.load('./results/Attention/pHLAIformer_{}_Length{}_Label_Unsoftmax_pepaatype_pepposition_NUM_pd.npy'.format(hla, length),
                                       allow_pickle = True).item()[new_label]
    except:
        print('No {} with {}, Use the overall attention for pepAAtype-peppsition'.format(hla, length))
        aatype_position = np.load('./results/Attention/pHLAIformer_Length_Label_Unsoftmax_pepaatype_pepposition_pd.npy',
                                  allow_pickle = True).item()[length][new_label]
        aatype_position_num = np.load('./results/Attention/pHLAIformer_Length_Label_Unsoftmax_pepaatype_pepposition_NUM_pd.npy',
                                      allow_pickle = True).item()[length][new_label]
    
    aatype_position.loc['sum'] = aatype_position.sum(axis = 0)
    aatype_position['sum'] = aatype_position.sum(axis = 1)
    
    return aatype_position, aatype_position_num

In [23]:
def HLA_aatype_position_contribution(aatype_position_pd, aatype_position_num_pd, length = 9):
    contrib = np.zeros((20, length))
    for aai, aa in enumerate(aatype_position_pd.index[:-1]): # 有个sum
        for pi, posi in enumerate(aatype_position_pd.columns[:-1]): # 有个sum
            
            p_aa_posi = aatype_position_pd.loc[aa, posi] / aatype_position_num_pd.loc[aa, posi]
            p_posi = aatype_position_num_pd.loc['sum', 'sum'] / aatype_position_pd.loc['sum', 'sum']
            
            contrib[aai, pi] = p_aa_posi * p_posi
            
    contrib = pd.DataFrame(contrib, index = aatype_position_pd.index[:-1], columns = aatype_position_pd.columns[:-1])
    contrib.fillna(0, inplace = True)
    return contrib

In [24]:
def HLA_Length_Label_pepaatype_pepposition(hla = False, length = 9, label = 1):
    aatype_position_pd, aatype_position_num_pd = HLA_length_aatype_position_num(hla, length, label)
    aatype_position_contrib_pd = HLA_aatype_position_contribution(aatype_position_pd, aatype_position_num_pd, length)
    return aatype_position_contrib_pd, aatype_position_pd, aatype_position_num_pd

In [25]:
def HLA_Length_position_keyaatype(aatype_position_pd, aatype_position_contrib_pd, length = 9):
    
    position_contrib_keyaatype = OrderedDict()
    for posi in range(1, length + 1):
        temp_sorted = aatype_position_contrib_pd[posi].sort_values(ascending = False)
        key_aatype = [k for k,v in OrderedDict(temp_sorted > 1).items() if v]
        position_contrib_keyaatype[posi] = [key_aatype, len(key_aatype), temp_sorted.max().round(2), temp_sorted.mean().round(2)]
    position_contrib_keyaatype = OrderedDict(sorted(position_contrib_keyaatype.items(), key = lambda t: (-t[1][2], t[1][1], -t[1][3])))
   
    ###################
    
    aatype_position_pd = aatype_position_pd.drop(index = 'sum')
    aatype_position_pd = aatype_position_pd.drop(columns = 'sum')
    position_keyaatype = OrderedDict()
    for posi in range(1, length + 1):
        temp_sorted = aatype_position_pd[posi].sort_values(ascending = False)
        key_aatype = [k for k,v in OrderedDict(temp_sorted > aatype_position_pd[posi].mean()).items() if v]
        position_keyaatype[posi] = [key_aatype, len(key_aatype), temp_sorted.max().round(2), temp_sorted.mean().round(2)]
    position_keyaatype = OrderedDict(sorted(position_keyaatype.items(), key = lambda t: (t[1][1], -t[1][2], t[1][3])))
    
    return position_contrib_keyaatype, position_keyaatype

In [26]:
def HLA_Length_aatype_position_contrib_attn_num(hla = False, length = 9, label = 1):
    aatype_position_contrib_pd, aatype_position_pd, aatype_position_num_pd = \
    HLA_Length_Label_pepaatype_pepposition(hla, length, label)

    position_contrib_keyaatype, position_keyaatype = \
    HLA_Length_position_keyaatype(aatype_position_pd, aatype_position_contrib_pd, length)
    
    return position_contrib_keyaatype, position_keyaatype, aatype_position_contrib_pd, aatype_position_pd, aatype_position_num_pd

In [27]:
def HLA_pHLA_contrib_keyaatype_attn_num(data, attn_data, hla = False, peptide = False, label = 1):
    
    length = len(peptide)

    ### HLA positive
    position_contrib_keyaatype, position_keyaatype, aatype_position_contrib_pd, aatype_position_pd, aatype_position_num_pd\
    = HLA_Length_aatype_position_contrib_attn_num(hla, length, label)

    ### pHLA
    pHLA_attns_pd, pHLA_keyaatype, pHLA_keyaatype_contrib = pHLA_attns_keyaatype_keyaacontrib(data, attn_data, hla, peptide)
    
    return position_contrib_keyaatype, position_keyaatype, aatype_position_contrib_pd, aatype_position_pd, aatype_position_num_pd, \
            pHLA_attns_pd, pHLA_keyaatype, pHLA_keyaatype_contrib

# 举例×
- peptide = 'SRELSFTSA'
- hla = 'HLA-A*68:01'
- peptide-HLA的attention vs HLA_Positive_attention
    - 9A 26.94 < 9RK 最大值587.98: 替换9A为9RK
    - 4L 20.63 < 2TVASIL 最大值200.97: 替换2R为2T
    - 4L 20.63 < 1ESDTNA 最大值189.10: 替换1S为1E
    - 假设4L 20.63 > 3ASIVFT 最大值16: 替换4L为4LAGDEPKQL中的A（因为L==L）
- 最多替换3个氨基酸以达到目的(不设置也行，反正最多也就替换14个氨基酸……还是先设置成3吧）
- 添加netMHCpan_BA的IC50预测值

In [303]:
# 最多4种氨基酸突变，共有120种：3+3*3+3*3*3+3*3*3*3

In [28]:
def oneposition_mut_peptides(mut_posi, mut_aatypes, peptide):

    mut_peptides = []
    for mut_aa in mut_aatypes:
        index = mut_posi - 1
        mut_peptide = peptide[:index] + mut_aa + peptide[index+1:]
        mut_peptides.append(mut_peptide)
    print(mut_peptides)
    return mut_peptides

In [29]:
def find_mutate_position_aatype(hla_oripeptide_all_mutpeptides):
    original_peptide = hla_oripeptide_all_mutpeptides[0]
    pep_length = len(original_peptide)
    mutate_position_aatype, mutate_num = [], []
    for pep in hla_oripeptide_all_mutpeptides:
        s = ''
        for i in range(pep_length):
            if original_peptide[i] != pep[i]:
                s += str(i+1) + '|' + original_peptide[i] + '/' + pep[i] + ','
        mutate_num.append(len(s[:-1].split(',')))
        mutate_position_aatype.append(s[:-1])
    return mutate_position_aatype, mutate_num

In [56]:
def mutation_positive_peptide_1(hla_peptide_keyaatype_contrib,
                              HLA_length_positive_position_contrib_keyaatype,
                              HLA_length_positive_position_keyaatype,
                              peptide = 'SRELSFTSA', hla = 'HLA-A*68:01'):
    # 用贡献值排序
    neg_mut_position_order = list(hla_peptide_keyaatype_contrib.keys())
    pos_mut_position_order = list(HLA_length_positive_position_contrib_keyaatype.keys())
    print(neg_mut_position_order, pos_mut_position_order)
    # 用贡献值对比，真实值排序
    mut_peptides_step = dict()
    mut_peptides_step[0] = peptide
    all_peptides = []
    
    i, pos_i, neg_i = 0, 0, 0
    mut_positions = []
    while i <= 3:
        
        for idx, item in enumerate(neg_mut_position_order):
            if item not in mut_positions:
                posi = item
                neg_i = idx
                break
        for idx, item in enumerate(pos_mut_position_order):
            if item not in mut_positions:
                pos_i = idx
                break
        print(mut_positions, neg_i, pos_i)
            
        (aatype, attnsum) = hla_peptide_keyaatype_contrib[posi]
        if attnsum < HLA_length_positive_position_contrib_keyaatype[pos_mut_position_order[pos_i]][3]: # 比较contrib的mean
            mut_posi = pos_mut_position_order[pos_i]
            mut_aatypes_all = HLA_length_positive_position_keyaatype[mut_posi][0] # 用真实值的aatype
            mut_aatypes = [aa for aai, aa in enumerate(mut_aatypes_all) if aai < 3]
            print('Effect of {}{} in Negative {} < Effect of {}{} in Positive {}, Replace to {}{}'.format(posi, aatype, peptide,mut_posi, mut_aatypes_all, hla, mut_posi, mut_aatypes))
        else:
            mut_posi = posi
            mut_aatypes_all = HLA_length_positive_position_keyaatype[mut_posi][0]
            mut_aatypes = [aa for aai, aa in enumerate(mut_aatypes_all) if aai < 3]
            print('Effect of {}{} in Negative {} > Effect of {}{} in Positive {}, Replace to {}{}'.format(posi, aatype, peptide, mut_posi, mut_aatypes_all, hla, mut_posi, mut_aatypes))

        if i == 0: 
            mut_peptides = oneposition_mut_peptides(mut_posi, mut_aatypes, peptide)
        else:
            mut_peptides_new = []
            for pep in mut_peptides:
                mut_peptides_new.extend(oneposition_mut_peptides(mut_posi, mut_aatypes, pep))
            mut_peptides = mut_peptides_new
        mut_peptides_step[i+1] = mut_peptides
        all_peptides.extend(mut_peptides)
         
        mut_positions.append(mut_posi)
        i += 1 

    return mut_peptides_step, all_peptides#' '.join(all_peptides_df.mutation_peptide), all_peptides_df

In [54]:
def mutation_positive_peptide_2(hla_peptide_keyaatype,
                              HLA_length_positive_position_keyaatype,
                              peptide = 'SRELSFTSA', hla = 'HLA-A*68:01'):
    # 用真实值排序
    neg_mut_position_order = list(hla_peptide_keyaatype.keys())
    pos_mut_position_order = list(HLA_length_positive_position_keyaatype.keys())
    print(neg_mut_position_order, pos_mut_position_order)
    # 用真实值对比：优先考虑正样本贡献（正样本是所有样本的累加值，所以可以认为一定大于负样本）
    mut_peptides_step = dict()
    mut_peptides_step[0] = peptide
    all_peptides = []
    
    i, pos_i, neg_i = 0, 0, 0
    mut_positions = []
    while i <= 3:
        
        for idx, item in enumerate(neg_mut_position_order):
            if item not in mut_positions:
                posi = item
                neg_i = idx
                break
        for idx, item in enumerate(pos_mut_position_order):
            if item not in mut_positions:
                pos_i = idx
                break
        print(mut_positions, neg_i, pos_i)
        
        (aatype, attnsum) = hla_peptide_keyaatype[posi]
        if attnsum < HLA_length_positive_position_keyaatype[pos_mut_position_order[pos_i]][3]: # 比较contrib的mean
            mut_posi = pos_mut_position_order[pos_i]
            mut_aatypes_all = HLA_length_positive_position_keyaatype[mut_posi][0] # 用真实值的aatype
            mut_aatypes = [aa for aai, aa in enumerate(mut_aatypes_all) if aai < 3]
            print('Effect of {}{} in Negative {} < Effect of {}{} in Positive {}, Replace to {}{}'.format(posi, aatype, peptide,mut_posi, mut_aatypes_all, hla, mut_posi, mut_aatypes))
#             pos_i += 1
        else:
            mut_posi = posi
            mut_aatypes_all = HLA_length_positive_position_keyaatype[mut_posi][0]
            mut_aatypes = [aa for aai, aa in enumerate(mut_aatypes_all) if aai < 3]
            print('Effect of {}{} in Negative {} > Effect of {}{} in Positive {}, Replace to {}{}'.format(posi, aatype, peptide, mut_posi, mut_aatypes_all, hla, mut_posi, mut_aatypes))
#             neg_i += 1

        if i == 0: 
            mut_peptides = oneposition_mut_peptides(mut_posi, mut_aatypes, peptide)
        else:
            mut_peptides_new = []
            for pep in mut_peptides:
                mut_peptides_new.extend(oneposition_mut_peptides(mut_posi, mut_aatypes, pep))
            mut_peptides = mut_peptides_new
        mut_peptides_step[i+1] = mut_peptides
        all_peptides.extend(mut_peptides)
         
        mut_positions.append(mut_posi)
        i += 1 

    return mut_peptides_step, all_peptides#

In [52]:
def mutation_positive_peptide_3(hla_peptide_keyaatype,
                              HLA_length_positive_position_keyaatype,
                              peptide = 'SRELSFTSA', hla = 'HLA-A*68:01'):
    # 用真实值排序
    neg_mut_position_order = list(hla_peptide_keyaatype.keys())
    pos_mut_position_order = list(HLA_length_positive_position_keyaatype.keys())
    print(neg_mut_position_order, pos_mut_position_order)
    # 用真实值对比：优先考虑负样本贡献（正样本是所有样本的累加值，所以可以认为一定大于负样本）
    mut_peptides_step = dict()
    mut_peptides_step[0] = peptide
    all_peptides = []
    
    i, pos_i, neg_i = 0, 0, 0
    mut_positions = []
    while i <= 3:
        for idx, item in enumerate(neg_mut_position_order):
            if item not in mut_positions:
                posi = item
                neg_i = idx
                break
        for idx, item in enumerate(pos_mut_position_order):
            if item not in mut_positions:
                pos_i = idx
                break
        print(mut_positions, neg_i, pos_i)
        (aatype, attnsum) = hla_peptide_keyaatype[posi]
        if i == 0 or attnsum > HLA_length_positive_position_keyaatype[pos_mut_position_order[i]][3]: # 比较contrib的mean
            mut_posi = posi
            mut_aatypes_all = HLA_length_positive_position_keyaatype[mut_posi][0]
            mut_aatypes = [aa for aai, aa in enumerate(mut_aatypes_all) if aai < 3]
            print('Effect of {}{} in Negative {} > Effect of {}{} in Positive {}, Replace to {}{}'.format(posi, aatype, peptide, mut_posi, mut_aatypes_all, hla, mut_posi, mut_aatypes))

        else:
            mut_posi = pos_mut_position_order[pos_i]
            mut_aatypes_all = HLA_length_positive_position_keyaatype[mut_posi][0] # 用真实值的aatype
            mut_aatypes = [aa for aai, aa in enumerate(mut_aatypes_all) if aai < 3]
            print('Effect of {}{} in Negative {} < Effect of {}{} in Positive {}, Replace to {}{}'.format(posi, aatype, peptide,mut_posi, mut_aatypes_all, hla, mut_posi, mut_aatypes))
            
        if i == 0: 
            mut_peptides = oneposition_mut_peptides(mut_posi, mut_aatypes, peptide)
        else:
            mut_peptides_new = []
            for pep in mut_peptides:
                mut_peptides_new.extend(oneposition_mut_peptides(mut_posi, mut_aatypes, pep))
            mut_peptides = mut_peptides_new
        mut_peptides_step[i+1] = mut_peptides
        all_peptides.extend(mut_peptides)
        
        mut_positions.append(mut_posi)
        i += 1        

    return mut_peptides_step, all_peptides#

In [38]:
def mutation_positive_peptide_4(hla_peptide_keyaatype,
                              HLA_length_positive_position_keyaatype,
                              peptide = 'SRELSFTSA', hla = 'HLA-A*68:01'):
    # 用真实值排序
    neg_mut_position_order = list(hla_peptide_keyaatype.keys())
    if len(peptide) == 8:
        pos_mut_position_order = [i for i in list(HLA_length_positive_position_keyaatype.keys()) if i in [1, 2, 8]]
    else:
        pos_mut_position_order = [i for i in list(HLA_length_positive_position_keyaatype.keys()) if i in [1, 2, 9]]
    print(neg_mut_position_order, pos_mut_position_order)
    # 改变129位+neg位
    mut_peptides_step = dict()
    mut_peptides_step[0] = peptide
    all_peptides = []
    pos_i, neg_i = -1, -1
    mut_positions = []
    i = 0
    while len(mut_positions) < 4:
        
        if i < 3: # 0 1 2
            mut_posi = pos_mut_position_order[i]
        elif i == 3:
            for item in neg_mut_position_order:
                if item not in mut_positions:
                    mut_posi = item
                    break
        
        (aatype, attnsum) = hla_peptide_keyaatype[mut_posi]
        mut_aatypes_all = HLA_length_positive_position_keyaatype[mut_posi][0] # 用真实值的aatype
        mut_aatypes = [aa for aai, aa in enumerate(mut_aatypes_all) if aai < 3]
        print('{}{} in Negative {} → {}{} in Positive {}'.format(mut_posi, aatype, peptide,mut_posi, mut_aatypes, hla))
            
        if i == 0: 
            mut_peptides = oneposition_mut_peptides(mut_posi, mut_aatypes, peptide)
        else:
            mut_peptides_new = []
            for pep in mut_peptides:
                mut_peptides_new.extend(oneposition_mut_peptides(mut_posi, mut_aatypes, pep))
            mut_peptides = mut_peptides_new
        mut_peptides_step[i+1] = mut_peptides
        all_peptides.extend(mut_peptides)
        
        mut_positions.append(mut_posi)
        i += 1
    return mut_peptides_step, all_peptides#

In [34]:
def pHLA_mutation_peptides(data, attn_data, idx = -1, hla = False, peptide = False):
    if not(peptide and hla) and idx > -1:
        hla = data.iloc[idx].HLA
        peptide = data.iloc[idx].peptide
        
    HLA_Length_Positive_position_contrib_keyaatype, HLA_Length_Positive_position_keyaatype, \
    HLA_Length_Positive_aatype_position_contrib_pd, HLA_Length_Positive_aatype_position_pd, \
    HLA_Length_Positive_aatype_position_num_pd, \
    pHLA_attns_pd, pHLA_keyaatype, pHLA_keyaatype_contrib = HLA_pHLA_contrib_keyaatype_attn_num(data, 
                                                                                                attns, 
                                                                                                hla, 
                                                                                                peptide, 
                                                                                                label = 1)

    mut_peptides = []
    print('********** Strategy 1 **********')
    pHLA_mut_peptides_step_1, pHLA_mut_peptides_1 = \
    mutation_positive_peptide_1(
        pHLA_keyaatype_contrib,
        HLA_Length_Positive_position_contrib_keyaatype,
        HLA_Length_Positive_position_keyaatype,
        peptide = peptide, hla = hla)
    print('********** Strategy 2 **********')
    pHLA_mut_peptides_step_2, pHLA_mut_peptides_2 = \
    mutation_positive_peptide_2(
        pHLA_keyaatype,
        HLA_Length_Positive_position_keyaatype,
        peptide = peptide, hla = hla)
    print('********** Strategy 3 **********')
    pHLA_mut_peptides_step_3, pHLA_mut_peptides_3 = \
    mutation_positive_peptide_3(
        pHLA_keyaatype,
        HLA_Length_Positive_position_keyaatype,
        peptide = peptide, hla = hla)
    print('********** Strategy 4 **********')
    pHLA_mut_peptides_step_4, pHLA_mut_peptides_4 = \
    mutation_positive_peptide_4(
        pHLA_keyaatype,
        HLA_Length_Positive_position_keyaatype,
        peptide = peptide, hla = hla)

    mut_peptides.extend(pHLA_mut_peptides_1)
    mut_peptides.extend(pHLA_mut_peptides_2)
    mut_peptides.extend(pHLA_mut_peptides_3)
    mut_peptides.extend(pHLA_mut_peptides_4)

    mut_peptides = list(set(mut_peptides))
    mut_peptides = [peptide] + mut_peptides

    mutate_position_aatype = find_mutate_position_aatype(mut_peptides)

    all_peptides_df = pd.DataFrame([[peptide] * len(mut_peptides),
                                    mut_peptides, 
                                    mutate_position_aatype[0],
                                    mutate_position_aatype[1],
                                    [difflib.SequenceMatcher(None, item ,peptide).ratio() for item in mut_peptides]],
        index = ['original_peptide', 'mutation_peptide', 'mutation_position_AAtype', 'mutation_AA_number', 'sequence similarity']).T.drop_duplicates().sort_values(by = 'mutation_AA_number').reset_index(drop = True)
    all_peptides_df['HLA'] = hla
    print('********** {} | {} → # Mutation peptides = {}'.format(hla, peptide, all_peptides_df.shape[0]))
    return ' '.join(all_peptides_df.mutation_peptide), all_peptides_df

##### try

In [57]:
A6801_SRELSFTSA_all_peptides_IEDBfmt, A6801_SRELSFTSA_all_peptides_df = pHLA_mutation_peptides(predict_data, attns, idx = 0)
print(A6801_SRELSFTSA_all_peptides_IEDBfmt)
A6801_SRELSFTSA_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 2720.24it/s]


********** Strategy 1 **********
[1, 6, 5, 2, 8, 3, 7, 4] [1, 3, 2, 6, 4, 8, 5, 7]
[] 0 0
Effect of 1K in Negative KIPLRCTA < Effect of 1['R', 'S', 'G', 'H', 'I', 'T', 'K'] in Positive HLA-B*27:05, Replace to 1['R', 'S', 'G']
['RIPLRCTA', 'SIPLRCTA', 'GIPLRCTA']
[1] 1 1
Effect of 6C in Negative KIPLRCTA < Effect of 3['D', 'F', 'L', 'R', 'Y', 'G'] in Positive HLA-B*27:05, Replace to 3['D', 'F', 'L']
['RIDLRCTA', 'RIFLRCTA', 'RILLRCTA']
['SIDLRCTA', 'SIFLRCTA', 'SILLRCTA']
['GIDLRCTA', 'GIFLRCTA', 'GILLRCTA']
[1, 3] 1 2
Effect of 6C in Negative KIPLRCTA < Effect of 2['R', 'F', 'L'] in Positive HLA-B*27:05, Replace to 2['R', 'F', 'L']
['RRDLRCTA', 'RFDLRCTA', 'RLDLRCTA']
['RRFLRCTA', 'RFFLRCTA', 'RLFLRCTA']
['RRLLRCTA', 'RFLLRCTA', 'RLLLRCTA']
['SRDLRCTA', 'SFDLRCTA', 'SLDLRCTA']
['SRFLRCTA', 'SFFLRCTA', 'SLFLRCTA']
['SRLLRCTA', 'SFLLRCTA', 'SLLLRCTA']
['GRDLRCTA', 'GFDLRCTA', 'GLDLRCTA']
['GRFLRCTA', 'GFFLRCTA', 'GLFLRCTA']
['GRLLRCTA', 'GFLLRCTA', 'GLLLRCTA']
[1, 3, 2] 1 3
Effect of 6C 

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity,HLA
0,KIPLRCTA,KIPLRCTA,,1,1.000,HLA-B*27:05
1,KIPLRCTA,KLPLRCTA,2|I/L,1,0.875,HLA-B*27:05
2,KIPLRCTA,RIPLRCTA,1|K/R,1,0.875,HLA-B*27:05
3,KIPLRCTA,SIPLRCTA,1|K/S,1,0.875,HLA-B*27:05
4,KIPLRCTA,KRPLRCTA,2|I/R,1,0.875,HLA-B*27:05
5,KIPLRCTA,KFPLRCTA,2|I/F,1,0.875,HLA-B*27:05
6,KIPLRCTA,GIPLRCTA,1|K/G,1,0.875,HLA-B*27:05
7,KIPLRCTA,SIDLRCTA,"1|K/S,3|P/D",2,0.750,HLA-B*27:05
8,KIPLRCTA,GLPLRCTA,"1|K/G,2|I/L",2,0.750,HLA-B*27:05
9,KIPLRCTA,KLDLRCTA,"2|I/L,3|P/D",2,0.750,HLA-B*27:05


##### peptide = 'SRELSFTSA', hla = 'HLA-A*68:01'

In [58]:
A6801_SRELSFTSA_all_peptides_IEDBfmt, A6801_SRELSFTSA_all_peptides_df = pHLA_mutation_peptides(predict_data, attns, hla = 'HLA-A*68:01', peptide = 'SRELSFTSA')
print(A6801_SRELSFTSA_all_peptides_IEDBfmt)
A6801_SRELSFTSA_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 2613.09it/s]

********** Strategy 1 **********
[2, 6, 3, 1, 4, 9, 8, 5, 7] [1, 2, 3, 8, 4, 6, 5, 7, 9]
[] 0 0
Effect of 2R in Negative SRELSFTSA < Effect of 1['E', 'D', 'S', 'T', 'A', 'N'] in Positive HLA-A*68:01, Replace to 1['E', 'D', 'S']
['ERELSFTSA', 'DRELSFTSA', 'SRELSFTSA']
[1] 0 1
Effect of 2R in Negative SRELSFTSA < Effect of 2['T', 'V', 'A', 'S', 'I', 'L'] in Positive HLA-A*68:01, Replace to 2['T', 'V', 'A']
['ETELSFTSA', 'EVELSFTSA', 'EAELSFTSA']
['DTELSFTSA', 'DVELSFTSA', 'DAELSFTSA']
['STELSFTSA', 'SVELSFTSA', 'SAELSFTSA']
[1, 2] 1 2
Effect of 6F in Negative SRELSFTSA < Effect of 3['A', 'S', 'I', 'V', 'F', 'T'] in Positive HLA-A*68:01, Replace to 3['A', 'S', 'I']
['ETALSFTSA', 'ETSLSFTSA', 'ETILSFTSA']
['EVALSFTSA', 'EVSLSFTSA', 'EVILSFTSA']
['EAALSFTSA', 'EASLSFTSA', 'EAILSFTSA']
['DTALSFTSA', 'DTSLSFTSA', 'DTILSFTSA']
['DVALSFTSA', 'DVSLSFTSA', 'DVILSFTSA']
['DAALSFTSA', 'DASLSFTSA', 'DAILSFTSA']
['STALSFTSA', 'STSLSFTSA', 'STILSFTSA']
['SVALSFTSA', 'SVSLSFTSA', 'SVILSFTSA']
['SAALSFT

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity,HLA
0,SRELSFTSA,SRELSFTSA,,1,1.000000,HLA-A*68:01
1,SRELSFTSA,STELSFTSA,2|R/T,1,0.888889,HLA-A*68:01
2,SRELSFTSA,ERELSFTSA,1|S/E,1,0.888889,HLA-A*68:01
3,SRELSFTSA,SAELSFTSA,2|R/A,1,0.888889,HLA-A*68:01
4,SRELSFTSA,SVELSFTSA,2|R/V,1,0.888889,HLA-A*68:01
5,SRELSFTSA,SRELSFTSR,9|A/R,1,0.888889,HLA-A*68:01
6,SRELSFTSA,DRELSFTSA,1|S/D,1,0.888889,HLA-A*68:01
7,SRELSFTSA,SRELSFTSK,9|A/K,1,0.888889,HLA-A*68:01
8,SRELSFTSA,EVELSFTSA,"1|S/E,2|R/V",2,0.777778,HLA-A*68:01
9,SRELSFTSA,STELSFTSK,"2|R/T,9|A/K",2,0.777778,HLA-A*68:01


##### peptide = 'GGGLGPVDV', hla = 'HLA-A*68:01'

In [61]:
A6801_GGGLGPVDV_all_peptides_IEDBfmt, A6801_GGGLGPVDV_all_peptides_df = pHLA_mutation_peptides(predict_data, attns, hla = 'HLA-A*68:01', peptide = 'GGGLGPVDV')
print(A6801_GGGLGPVDV_all_peptides_IEDBfmt)
A6801_GGGLGPVDV_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 2763.65it/s]

********** Strategy 1 **********
[8, 1, 2, 3, 4, 5, 9, 6, 7] [1, 2, 3, 8, 4, 6, 5, 7, 9]
[] 0 0
Effect of 8D in Negative GGGLGPVDV < Effect of 1['E', 'D', 'S', 'T', 'A', 'N'] in Positive HLA-A*68:01, Replace to 1['E', 'D', 'S']
['EGGLGPVDV', 'DGGLGPVDV', 'SGGLGPVDV']
[1] 0 1
Effect of 8D in Negative GGGLGPVDV < Effect of 2['T', 'V', 'A', 'S', 'I', 'L'] in Positive HLA-A*68:01, Replace to 2['T', 'V', 'A']
['ETGLGPVDV', 'EVGLGPVDV', 'EAGLGPVDV']
['DTGLGPVDV', 'DVGLGPVDV', 'DAGLGPVDV']
['STGLGPVDV', 'SVGLGPVDV', 'SAGLGPVDV']
[1, 2] 0 2
Effect of 8D in Negative GGGLGPVDV < Effect of 3['A', 'S', 'I', 'V', 'F', 'T'] in Positive HLA-A*68:01, Replace to 3['A', 'S', 'I']
['ETALGPVDV', 'ETSLGPVDV', 'ETILGPVDV']
['EVALGPVDV', 'EVSLGPVDV', 'EVILGPVDV']
['EAALGPVDV', 'EASLGPVDV', 'EAILGPVDV']
['DTALGPVDV', 'DTSLGPVDV', 'DTILGPVDV']
['DVALGPVDV', 'DVSLGPVDV', 'DVILGPVDV']
['DAALGPVDV', 'DASLGPVDV', 'DAILGPVDV']
['STALGPVDV', 'STSLGPVDV', 'STILGPVDV']
['SVALGPVDV', 'SVSLGPVDV', 'SVILGPVDV']
['SAALGPV

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity,HLA
0,GGGLGPVDV,GGGLGPVDV,,1,1.000000,HLA-A*68:01
1,GGGLGPVDV,GGGLGPVDK,9|V/K,1,0.888889,HLA-A*68:01
2,GGGLGPVDV,GGGLGPVDR,9|V/R,1,0.888889,HLA-A*68:01
3,GGGLGPVDV,SGGLGPVDV,1|G/S,1,0.888889,HLA-A*68:01
4,GGGLGPVDV,GGGLGPVKV,8|D/K,1,0.888889,HLA-A*68:01
5,GGGLGPVDV,EGGLGPVDV,1|G/E,1,0.888889,HLA-A*68:01
6,GGGLGPVDV,DGGLGPVDV,1|G/D,1,0.888889,HLA-A*68:01
7,GGGLGPVDV,GGGLGPVQV,8|D/Q,1,0.888889,HLA-A*68:01
8,GGGLGPVDV,GGGLGPVLV,8|D/L,1,0.888889,HLA-A*68:01
9,GGGLGPVDV,DGGLGPVDR,"1|G/D,9|V/R",2,0.777778,HLA-A*68:01


##### peptide = 'AERLQENLQ', hla = 'HLA-A*68:01'

In [62]:
A6801_AERLQENLQ_all_peptides_IEDBfmt, A6801_AERLQENLQ_all_peptides_df = pHLA_mutation_peptides(predict_data, attns, hla = 'HLA-A*68:01', peptide = 'AERLQENLQ')
print(A6801_AERLQENLQ_all_peptides_IEDBfmt)
A6801_AERLQENLQ_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 2685.79it/s]

********** Strategy 1 **********
[2, 3, 7, 1, 8, 6, 4, 9, 5] [1, 2, 3, 8, 4, 6, 5, 7, 9]
[] 0 0
Effect of 2E in Negative AERLQENLQ < Effect of 1['E', 'D', 'S', 'T', 'A', 'N'] in Positive HLA-A*68:01, Replace to 1['E', 'D', 'S']
['EERLQENLQ', 'DERLQENLQ', 'SERLQENLQ']
[1] 0 1
Effect of 2E in Negative AERLQENLQ < Effect of 2['T', 'V', 'A', 'S', 'I', 'L'] in Positive HLA-A*68:01, Replace to 2['T', 'V', 'A']
['ETRLQENLQ', 'EVRLQENLQ', 'EARLQENLQ']
['DTRLQENLQ', 'DVRLQENLQ', 'DARLQENLQ']
['STRLQENLQ', 'SVRLQENLQ', 'SARLQENLQ']
[1, 2] 1 2
Effect of 3R in Negative AERLQENLQ < Effect of 3['A', 'S', 'I', 'V', 'F', 'T'] in Positive HLA-A*68:01, Replace to 3['A', 'S', 'I']
['ETALQENLQ', 'ETSLQENLQ', 'ETILQENLQ']
['EVALQENLQ', 'EVSLQENLQ', 'EVILQENLQ']
['EAALQENLQ', 'EASLQENLQ', 'EAILQENLQ']
['DTALQENLQ', 'DTSLQENLQ', 'DTILQENLQ']
['DVALQENLQ', 'DVSLQENLQ', 'DVILQENLQ']
['DAALQENLQ', 'DASLQENLQ', 'DAILQENLQ']
['STALQENLQ', 'STSLQENLQ', 'STILQENLQ']
['SVALQENLQ', 'SVSLQENLQ', 'SVILQENLQ']
['SAALQEN

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity,HLA
0,AERLQENLQ,AERLQENLQ,,1,1.000000,HLA-A*68:01
1,AERLQENLQ,AVRLQENLQ,2|E/V,1,0.888889,HLA-A*68:01
2,AERLQENLQ,AERLQENLR,9|Q/R,1,0.888889,HLA-A*68:01
3,AERLQENLQ,SERLQENLQ,1|A/S,1,0.888889,HLA-A*68:01
4,AERLQENLQ,EERLQENLQ,1|A/E,1,0.888889,HLA-A*68:01
5,AERLQENLQ,AARLQENLQ,2|E/A,1,0.888889,HLA-A*68:01
6,AERLQENLQ,ATRLQENLQ,2|E/T,1,0.888889,HLA-A*68:01
7,AERLQENLQ,DERLQENLQ,1|A/D,1,0.888889,HLA-A*68:01
8,AERLQENLQ,AERLQENLK,9|Q/K,1,0.888889,HLA-A*68:01
9,AERLQENLQ,EERLQENLK,"1|A/E,9|Q/K",2,0.777778,HLA-A*68:01


##### peptide = 'KLVSIKIQML', hla = 'HLA-A*68:01'

In [151]:
A6801_KLVSIKIQML_all_peptides_IEDBfmt, A6801_KLVSIKIQML_all_peptides_df = pHLA_mutation_peptides('HLA-A*68:01', 'KLVSIKIQML', data, attn_data)
print(A6801_KLVSIKIQML_all_peptides_IEDBfmt)
A6801_KLVSIKIQML_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 8815.68it/s]

********** Strategy 1 **********
[1, 9, 2, 10, 7, 6, 5, 4, 8, 3] [1, 10, 2, 3, 4, 9, 8, 7, 5, 6]
Effect of 1K in Negative KLVSIKIQML < Effect of 1['E', 'S', 'D', 'T', 'A', 'I', 'N', 'L', 'F'] in Positive HLA-A*68:01, Replace to 1['E', 'S', 'D']
['ELVSIKIQML', 'SLVSIKIQML', 'DLVSIKIQML']
Effect of 1K in Negative KLVSIKIQML < Effect of 10['K', 'R', 'Y'] in Positive HLA-A*68:01, Replace to 10['K', 'R', 'Y']
['ELVSIKIQMK', 'ELVSIKIQMR', 'ELVSIKIQMY']
['SLVSIKIQMK', 'SLVSIKIQMR', 'SLVSIKIQMY']
['DLVSIKIQMK', 'DLVSIKIQMR', 'DLVSIKIQMY']
Effect of 1K in Negative KLVSIKIQML < Effect of 2['T', 'V', 'S', 'L', 'A', 'I'] in Positive HLA-A*68:01, Replace to 2['T', 'V', 'S']
['ETVSIKIQMK', 'EVVSIKIQMK', 'ESVSIKIQMK']
['ETVSIKIQMR', 'EVVSIKIQMR', 'ESVSIKIQMR']
['ETVSIKIQMY', 'EVVSIKIQMY', 'ESVSIKIQMY']
['STVSIKIQMK', 'SVVSIKIQMK', 'SSVSIKIQMK']
['STVSIKIQMR', 'SVVSIKIQMR', 'SSVSIKIQMR']
['STVSIKIQMY', 'SVVSIKIQMY', 'SSVSIKIQMY']
['DTVSIKIQMK', 'DVVSIKIQMK', 'DSVSIKIQMK']
['DTVSIKIQMR', 'DVVSIKIQMR', 

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity
0,KLVSIKIQML,KLVSIKIQML,,1,1.0
1,KLVSIKIQML,DLVSIKIQML,1|K/D,1,0.9
2,KLVSIKIQML,KTVSIKIQML,2|L/T,1,0.9
3,KLVSIKIQML,SLVSIKIQML,1|K/S,1,0.9
4,KLVSIKIQML,KLVSIKIQMK,10|L/K,1,0.9
5,KLVSIKIQML,KSVSIKIQML,2|L/S,1,0.9
6,KLVSIKIQML,KLVSIKIQMY,10|L/Y,1,0.9
7,KLVSIKIQML,KLVSIKIQMR,10|L/R,1,0.9
8,KLVSIKIQML,KVVSIKIQML,2|L/V,1,0.9
9,KLVSIKIQML,ELVSIKIQML,1|K/E,1,0.9


##### peptide = 'QVIIFVKSVQR', hla = 'HLA-A*68:01'
正样本，长度为11的肽没有C

In [150]:
A6801_QVIIFVKSVQR_all_peptides_IEDBfmt, A6801_QVIIFVKSVQR_all_peptides_df = pHLA_mutation_peptides('HLA-A*68:01', 'QVIIFVKSVQR', data, attn_data)
print(A6801_QVIIFVKSVQR_all_peptides_IEDBfmt)
A6801_QVIIFVKSVQR_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 11128.75it/s]

********** Strategy 1 **********
[11, 5, 3, 7, 4, 8, 2, 1, 10, 9, 6] [2, 1, 11, 3, 5, 8, 10, 6, 4, 9, 7]
Effect of 11R in Negative QVIIFVKSVQR < Effect of 2['V', 'T', 'A', 'S', 'I'] in Positive HLA-A*68:01, Replace to 2['V', 'T', 'A']
['QVIIFVKSVQR', 'QTIIFVKSVQR', 'QAIIFVKSVQR']
Effect of 11R in Negative QVIIFVKSVQR < Effect of 1['E', 'D', 'S', 'T'] in Positive HLA-A*68:01, Replace to 1['E', 'D', 'S']
['EVIIFVKSVQR', 'DVIIFVKSVQR', 'SVIIFVKSVQR']
['ETIIFVKSVQR', 'DTIIFVKSVQR', 'STIIFVKSVQR']
['EAIIFVKSVQR', 'DAIIFVKSVQR', 'SAIIFVKSVQR']
Effect of 11R in Negative QVIIFVKSVQR > Effect of 11['R', 'K'] in Positive HLA-A*68:01, Replace to 11['R', 'K']
['EVIIFVKSVQR', 'EVIIFVKSVQK']
['DVIIFVKSVQR', 'DVIIFVKSVQK']
['SVIIFVKSVQR', 'SVIIFVKSVQK']
['ETIIFVKSVQR', 'ETIIFVKSVQK']
['DTIIFVKSVQR', 'DTIIFVKSVQK']
['STIIFVKSVQR', 'STIIFVKSVQK']
['EAIIFVKSVQR', 'EAIIFVKSVQK']
['DAIIFVKSVQR', 'DAIIFVKSVQK']
['SAIIFVKSVQR', 'SAIIFVKSVQK']
Effect of 5F in Negative QVIIFVKSVQR > Effect of 5['V', 'A', 'P',

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity
0,QVIIFVKSVQR,QVIIFVKSVQR,,1,1.000000
1,QVIIFVKSVQR,DVIIFVKSVQR,1|Q/D,1,0.909091
2,QVIIFVKSVQR,SVIIFVKSVQR,1|Q/S,1,0.909091
3,QVIIFVKSVQR,QTIIFVKSVQR,2|V/T,1,0.909091
4,QVIIFVKSVQR,QAIIFVKSVQR,2|V/A,1,0.909091
5,QVIIFVKSVQR,EVIIFVKSVQR,1|Q/E,1,0.909091
6,QVIIFVKSVQR,QVIIFVKSVQK,11|R/K,1,0.909091
7,QVIIFVKSVQR,DVIIFVKSVQK,"1|Q/D,11|R/K",2,0.818182
8,QVIIFVKSVQR,DAIIFVKSVQR,"1|Q/D,2|V/A",2,0.818182
9,QVIIFVKSVQR,SVIIVVKSVQR,"1|Q/S,5|F/V",2,0.818182


##### HLA-C*16:01, VSPMRQVEPPA
长度11，负样本

In [145]:
C1601_VSPMRQVEPPA_all_peptides_IEDBfmt, C1601_VSPMRQVEPPA_all_peptides_df = pHLA_mutation_peptides('HLA-C*16:01', 'VSPMRQVEPPA', data, attn_data)
print(C1601_VSPMRQVEPPA_all_peptides_IEDBfmt)
C1601_VSPMRQVEPPA_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 8905.10it/s]


********** Strategy 1 **********
[3, 5, 4, 11, 6, 2, 8, 10, 1, 9, 7] [1, 2, 3, 11, 4, 5, 7, 9, 10, 8, 6]
Effect of 3P in Negative VSPMRQVEPPA < Effect of 1['V', 'Y', 'I', 'A', 'R', 'K', 'F', 'G', 'L', 'S'] in Positive HLA-C*16:01, Replace to 1['V', 'Y', 'I']
['VSPMRQVEPPA', 'YSPMRQVEPPA', 'ISPMRQVEPPA']
Effect of 3P in Negative VSPMRQVEPPA < Effect of 2['A', 'S', 'T', 'M', 'I'] in Positive HLA-C*16:01, Replace to 2['A', 'S', 'T']
['VAPMRQVEPPA', 'VSPMRQVEPPA', 'VTPMRQVEPPA']
['YAPMRQVEPPA', 'YSPMRQVEPPA', 'YTPMRQVEPPA']
['IAPMRQVEPPA', 'ISPMRQVEPPA', 'ITPMRQVEPPA']
Effect of 3P in Negative VSPMRQVEPPA < Effect of 3['S', 'V', 'Y', 'A', 'I', 'T', 'L'] in Positive HLA-C*16:01, Replace to 3['S', 'V', 'Y']
['VASMRQVEPPA', 'VAVMRQVEPPA', 'VAYMRQVEPPA']
['VSSMRQVEPPA', 'VSVMRQVEPPA', 'VSYMRQVEPPA']
['VTSMRQVEPPA', 'VTVMRQVEPPA', 'VTYMRQVEPPA']
['YASMRQVEPPA', 'YAVMRQVEPPA', 'YAYMRQVEPPA']
['YSSMRQVEPPA', 'YSVMRQVEPPA', 'YSYMRQVEPPA']
['YTSMRQVEPPA', 'YTVMRQVEPPA', 'YTYMRQVEPPA']
['IASMRQVEPPA

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity
0,VSPMRQVEPPA,VSPMRQVEPPA,,1,1.000000
1,VSPMRQVEPPA,VSPMRQVERPA,9|P/R,1,0.909091
2,VSPMRQVEPPA,VSSMRQVEPPA,3|P/S,1,0.909091
3,VSPMRQVEPPA,VSYMRQVEPPA,3|P/Y,1,0.909091
4,VSPMRQVEPPA,VSPMRQVEIPA,9|P/I,1,0.909091
5,VSPMRQVEPPA,VSVMRQVEPPA,3|P/V,1,0.909091
6,VSPMRQVEPPA,ISPMRQVEPPA,1|V/I,1,0.909091
7,VSPMRQVEPPA,VSPMRQVEQPA,9|P/Q,1,0.909091
8,VSPMRQVEPPA,VAPMRQVEPPA,2|S/A,1,0.909091
9,VSPMRQVEPPA,YSPMRQVEPPA,1|V/Y,1,0.909091


##### 'HLA-B*27:02', 'NGKPLPGATPAK'，12

In [146]:
B2702_NGKPLPGATPAK_all_peptides_IEDBfmt, B2702_NGKPLPGATPAK_all_peptides_df = pHLA_mutation_peptides('HLA-B*27:02', 'NGKPLPGATPAK', data, attn_data)
print(B2702_NGKPLPGATPAK_all_peptides_IEDBfmt)
B2702_NGKPLPGATPAK_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 8569.52it/s]


********** Strategy 1 **********
[12, 1, 5, 7, 10, 2, 3, 11, 9, 8, 6, 4] [1, 12, 11, 10, 2, 3, 9, 8, 5, 4, 7, 6]
Effect of 12K in Negative NGKPLPGATPAK < Effect of 1['G', 'S', 'R', 'A', 'K', 'Q', 'I', 'T'] in Positive HLA-B*27:02, Replace to 1['G', 'S', 'R']
['GGKPLPGATPAK', 'SGKPLPGATPAK', 'RGKPLPGATPAK']
Effect of 12K in Negative NGKPLPGATPAK < Effect of 12['F', 'Y', 'W', 'L'] in Positive HLA-B*27:02, Replace to 12['F', 'Y', 'W']
['GGKPLPGATPAF', 'GGKPLPGATPAY', 'GGKPLPGATPAW']
['SGKPLPGATPAF', 'SGKPLPGATPAY', 'SGKPLPGATPAW']
['RGKPLPGATPAF', 'RGKPLPGATPAY', 'RGKPLPGATPAW']
Effect of 12K in Negative NGKPLPGATPAK < Effect of 11['L', 'S', 'G', 'T', 'Q', 'E', 'A', 'V'] in Positive HLA-B*27:02, Replace to 11['L', 'S', 'G']
['GGKPLPGATPLF', 'GGKPLPGATPSF', 'GGKPLPGATPGF']
['GGKPLPGATPLY', 'GGKPLPGATPSY', 'GGKPLPGATPGY']
['GGKPLPGATPLW', 'GGKPLPGATPSW', 'GGKPLPGATPGW']
['SGKPLPGATPLF', 'SGKPLPGATPSF', 'SGKPLPGATPGF']
['SGKPLPGATPLY', 'SGKPLPGATPSY', 'SGKPLPGATPGY']
['SGKPLPGATPLW', 'SGKPLP

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity
0,NGKPLPGATPAK,NGKPLPGATPAK,,1,1.000000
1,NGKPLPGATPAK,GGKPLPGATPAK,1|N/G,1,0.916667
2,NGKPLPGATPAK,NGKPLPGATPAF,12|K/F,1,0.916667
3,NGKPLPGATPAK,RGKPLPGATPAK,1|N/R,1,0.916667
4,NGKPLPGATPAK,NGKPLPGATPAW,12|K/W,1,0.916667
5,NGKPLPGATPAK,NGKPLPGATPAY,12|K/Y,1,0.916667
6,NGKPLPGATPAK,SGKPLPGATPAK,1|N/S,1,0.916667
7,NGKPLPGATPAK,NRKPLPGATPAK,2|G/R,1,0.916667
8,NGKPLPGATPAK,SRKPLPGATPAK,"1|N/S,2|G/R",2,0.833333
9,NGKPLPGATPAK,RGKPLPGATPAF,"1|N/R,12|K/F",2,0.833333


##### SLYGGNAVVELVTV	14	HLA-A*02:01 Positive

In [147]:
A0201_SLYGGNAVVELVTV_all_peptides_IEDBfmt, A0201_SLYGGNAVVELVTV_all_peptides_df = pHLA_mutation_peptides('HLA-A*02:01', 'SLYGGNAVVELVTV', data, attn_data)
print(A0201_SLYGGNAVVELVTV_all_peptides_IEDBfmt)
A0201_SLYGGNAVVELVTV_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 8888.33it/s]

********** Strategy 1 **********
[10, 2, 3, 11, 1, 4, 5, 6, 13, 7, 8, 9, 12, 14] [1, 2, 3, 14, 12, 11, 13, 6, 5, 4, 10, 9, 7, 8]
Effect of 10E in Negative SLYGGNAVVELVTV < Effect of 1['F', 'S', 'G', 'A', 'Y', 'V', 'I', 'K'] in Positive HLA-A*02:01, Replace to 1['F', 'S', 'G']
['FLYGGNAVVELVTV', 'SLYGGNAVVELVTV', 'GLYGGNAVVELVTV']
Effect of 10E in Negative SLYGGNAVVELVTV < Effect of 2['L'] in Positive HLA-A*02:01, Replace to 2['L']
['FLYGGNAVVELVTV']
['SLYGGNAVVELVTV']
['GLYGGNAVVELVTV']
Effect of 10E in Negative SLYGGNAVVELVTV < Effect of 3['F', 'Y', 'L', 'A', 'M', 'W', 'I'] in Positive HLA-A*02:01, Replace to 3['F', 'Y', 'L']
['FLFGGNAVVELVTV', 'FLYGGNAVVELVTV', 'FLLGGNAVVELVTV']
['SLFGGNAVVELVTV', 'SLYGGNAVVELVTV', 'SLLGGNAVVELVTV']
['GLFGGNAVVELVTV', 'GLYGGNAVVELVTV', 'GLLGGNAVVELVTV']
Effect of 10E in Negative SLYGGNAVVELVTV < Effect of 14['L', 'V', 'A', 'S'] in Positive HLA-A*02:01, Replace to 14['L', 'V', 'A']
['FLFGGNAVVELVTL', 'FLFGGNAVVELVTV', 'FLFGGNAVVELVTA']
['FLYGGNAVVELVT

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity
0,SLYGGNAVVELVTV,SLYGGNAVVELVTV,,1,1.000000
1,SLYGGNAVVELVTV,FLYGGNAVVELVTV,1|S/F,1,0.928571
2,SLYGGNAVVELVTV,SLYGGNAVEELVTV,9|V/E,1,0.928571
3,SLYGGNAVVELVTV,GLYGGNAVVELVTV,1|S/G,1,0.928571
4,SLYGGNAVVELVTV,SLLGGNAVVELVTV,3|Y/L,1,0.928571
5,SLYGGNAVVELVTV,SLYGGNAVVELVTL,14|V/L,1,0.928571
6,SLYGGNAVVELVTV,SLYGGNAVVEEVTV,11|L/E,1,0.928571
7,SLYGGNAVVELVTV,SLYGGNAVVLLVTV,10|E/L,1,0.928571
8,SLYGGNAVVELVTV,SLFGGNAVVELVTV,3|Y/F,1,0.928571
9,SLYGGNAVVELVTV,SLYGGNAVGELVTV,9|V/G,1,0.928571


##### AKEMMAQKRK	10	HLA-A*02:06	0

In [148]:
A0206_SLYGGNAVVELVTV_all_peptides_IEDBfmt, A0206_SLYGGNAVVELVTV_all_peptides_df = pHLA_mutation_peptides('HLA-A*02:06', 'AKEMMAQKRK', data, attn_data)
print(A0206_SLYGGNAVVELVTV_all_peptides_IEDBfmt)
A0206_SLYGGNAVVELVTV_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 9164.54it/s]


********** Strategy 1 **********
[4, 3, 9, 10, 5, 1, 2, 6, 7, 8] [2, 1, 3, 10, 9, 4, 5, 7, 8, 6]
Effect of 4M in Negative AKEMMAQKRK < Effect of 2['L', 'I', 'V', 'M', 'T', 'A', 'Q'] in Positive HLA-A*02:06, Replace to 2['L', 'I', 'V']
['ALEMMAQKRK', 'AIEMMAQKRK', 'AVEMMAQKRK']
Effect of 4M in Negative AKEMMAQKRK < Effect of 1['F', 'L', 'I', 'S', 'K', 'Y', 'A', 'T'] in Positive HLA-A*02:06, Replace to 1['F', 'L', 'I']
['FLEMMAQKRK', 'LLEMMAQKRK', 'ILEMMAQKRK']
['FIEMMAQKRK', 'LIEMMAQKRK', 'IIEMMAQKRK']
['FVEMMAQKRK', 'LVEMMAQKRK', 'IVEMMAQKRK']
Effect of 4M in Negative AKEMMAQKRK < Effect of 3['L', 'I', 'V', 'M', 'F', 'S', 'Y', 'A'] in Positive HLA-A*02:06, Replace to 3['L', 'I', 'V']
['FLLMMAQKRK', 'FLIMMAQKRK', 'FLVMMAQKRK']
['LLLMMAQKRK', 'LLIMMAQKRK', 'LLVMMAQKRK']
['ILLMMAQKRK', 'ILIMMAQKRK', 'ILVMMAQKRK']
['FILMMAQKRK', 'FIIMMAQKRK', 'FIVMMAQKRK']
['LILMMAQKRK', 'LIIMMAQKRK', 'LIVMMAQKRK']
['IILMMAQKRK', 'IIIMMAQKRK', 'IIVMMAQKRK']
['FVLMMAQKRK', 'FVIMMAQKRK', 'FVVMMAQKRK']
['LVLM

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity
0,AKEMMAQKRK,AKEMMAQKRK,,1,1.0
1,AKEMMAQKRK,AKESMAQKRK,4|M/S,1,0.9
2,AKEMMAQKRK,ALEMMAQKRK,2|K/L,1,0.9
3,AKEMMAQKRK,AVEMMAQKRK,2|K/V,1,0.9
4,AKEMMAQKRK,AKEMMAQKRI,10|K/I,1,0.9
5,AKEMMAQKRK,AKEMMAQKRL,10|K/L,1,0.9
6,AKEMMAQKRK,AKEGMAQKRK,4|M/G,1,0.9
7,AKEMMAQKRK,AKEMMAQKRV,10|K/V,1,0.9
8,AKEMMAQKRK,AIEMMAQKRK,2|K/I,1,0.9
9,AKEMMAQKRK,AKELMAQKRK,4|M/L,1,0.9


##### VKKIKEHVRSKTKV	14	HLA-B*27:05	0	

In [149]:
B2705_VKKIKEHVRSKTKV_all_peptides_IEDBfmt, B2705_VKKIKEHVRSKTKV_all_peptides_df = pHLA_mutation_peptides('HLA-B*27:05', 'VKKIKEHVRSKTKV', data, attn_data)
print(B2705_VKKIKEHVRSKTKV_all_peptides_IEDBfmt)
B2705_VKKIKEHVRSKTKV_all_peptides_df

100%|██████████| 9/9 [00:00<00:00, 7834.94it/s]


********** Strategy 1 **********
[13, 11, 9, 2, 1, 7, 6, 8, 3, 4, 5, 10, 14, 12] [2, 1, 13, 12, 11, 14, 3, 9, 10, 5, 6, 4, 8, 7]
Effect of 13K in Negative VKKIKEHVRSKTKV < Effect of 2['R', 'G', 'V', 'S', 'P', 'L', 'E', 'D', 'A'] in Positive HLA-B*27:05, Replace to 2['R', 'G', 'V']
['VRKIKEHVRSKTKV', 'VGKIKEHVRSKTKV', 'VVKIKEHVRSKTKV']
Effect of 13K in Negative VKKIKEHVRSKTKV < Effect of 1['S', 'T', 'H', 'G', 'A', 'R'] in Positive HLA-B*27:05, Replace to 1['S', 'T', 'H']
['SRKIKEHVRSKTKV', 'TRKIKEHVRSKTKV', 'HRKIKEHVRSKTKV']
['SGKIKEHVRSKTKV', 'TGKIKEHVRSKTKV', 'HGKIKEHVRSKTKV']
['SVKIKEHVRSKTKV', 'TVKIKEHVRSKTKV', 'HVKIKEHVRSKTKV']
Effect of 13K in Negative VKKIKEHVRSKTKV < Effect of 13['S', 'G', 'P', 'K', 'L', 'E', 'V', 'A', 'T', 'Q', 'R'] in Positive HLA-B*27:05, Replace to 13['S', 'G', 'P']
['SRKIKEHVRSKTSV', 'SRKIKEHVRSKTGV', 'SRKIKEHVRSKTPV']
['TRKIKEHVRSKTSV', 'TRKIKEHVRSKTGV', 'TRKIKEHVRSKTPV']
['HRKIKEHVRSKTSV', 'HRKIKEHVRSKTGV', 'HRKIKEHVRSKTPV']
['SGKIKEHVRSKTSV', 'SGKIKEHVRS

,original_peptide,mutation_peptide,mutation_position_AAtype,mutation_AA_number,sequence similarity
0,VKKIKEHVRSKTKV,VKKIKEHVRSKTKV,,1,1.000000
1,VKKIKEHVRSKTKV,VRKIKEHVRSKTKV,2|K/R,1,0.928571
2,VKKIKEHVRSKTKV,VKKIKEHVRSKTGV,13|K/G,1,0.928571
3,VKKIKEHVRSKTKV,VKKIKEHVRSKTPV,13|K/P,1,0.928571
4,VKKIKEHVRSKTKV,SKKIKEHVRSKTKV,1|V/S,1,0.928571
5,VKKIKEHVRSKTKV,VKKIKEHVRSKTSV,13|K/S,1,0.928571
6,VKKIKEHVRSKTKV,HKKIKEHVRSKTKV,1|V/H,1,0.928571
7,VKKIKEHVRSKTKV,VGKIKEHVRSKTKV,2|K/G,1,0.928571
8,VKKIKEHVRSKTKV,TKKIKEHVRSKTKV,1|V/T,1,0.928571
9,VKKIKEHVRSKTKV,VVKIKEHVRSKTKV,2|K/V,1,0.928571
